![Autodistill banner](https://camo.githubusercontent.com/2a4712931b5698aca7a95dfd90eb56a693fe59320eee64c1897042d5c3c21959/68747470733a2f2f6d656469612e726f626f666c6f772e636f6d2f6f70656e2d736f757263652f6175746f64697374696c6c2f6175746f64697374696c6c2d62616e6e65722e706e67)

# Create a Shipping Container Detection Model Without Labelling

In this notebook, we're going to create a model that can detect shipping containers. We will employ a technique called distillation, where we use the knowledge in one model -- Grounding DINO, a zero-shot object detection model -- to build a smaller model that is tuned to a specific use case.

By distilling Grounding DINO, we can create a new model that is smaller, more performant in terms of inference time, and easier to tune because the model is smaller and we have the full underlying dataset used to train our model.

To distill the knowledge from Grounding DINO into a smaller model specifically for detecting shipping containers, we will use `autodistill`, a new Python package for distilling large computer vision models into smaller ones. We'll train a new model using YOLOv8 and our labelled images.

By the end of this guide, we'll have a shipping container detection model that we can run on images and videos.

Without further ado, let's start coding!

## Setup

First, let's install and import the dependencies required for this project and set the requisite configuration values for our project. We'll also load the shipping container dataset for use in our project.

In [9]:
!pip install -q autodistill autodistill-grounded-sam autodistill-yolov8 supervision
!wget https://autodistill.s3.amazonaws.com/containers.zip
!unzip -q containers.zip -d containers

In [10]:
from autodistill_grounded_sam import GroundedSAM
from autodistill.detection import CaptionOntology
from autodistill_yolov8 import YOLOv8
import supervision as sv
import cv2
import os

HOME = os.getcwd()
DATA = HOME + "/data/"

## Label the Container Dataset with Grounding DINO

Now that we have installed the required dependencies and configured our project, we're ready to start building our model.

The first step is to label all the cars in the container images in our dataset. To do so, we'll leverage `autodistill` to programatically label images using Grounding DINO.

The steps we'll follow are:

1. Create an ontology for labelling images
2. Install Grounding DINO
3. Run inference on Grounding DINO using images from our dataset

This will give us a folder of annotated images we can use to train a YOLOv8 model.

In [ ]:
base_model = GroundedSAM(ontology=CaptionOntology({"shipping container": "container"}))

In [ ]:
# label all of our images
base_model.label("./context_images", extension=".jpeg")

## Train a YOLOv8 Model

Next, we'll use our labelled images to train a YOLOv8 model to detect shipping containers. To train the model, we'll use the Python methods available in the `ultralytics` pip package. To learn more about training your own YOLOv8 model, check out our [How to Train a YOLOv8 Object Detection Model notebook](https://github.com/roboflow-ai/notebooks/blob/main/notebooks/train-yolov8-object-detection-on-custom-dataset.ipynb).

We'll save our weights to disk so you can use your weights wherever you would like to deploy your model.

In [ ]:
target_model = YOLOv8("yolov8n.pt")

target_model.train("./containers_labeled/data.yaml", epochs=200)

# export weights for future use
saved_weights = target_model.export(format="onnx")

In [ ]:
# Calculate performance metrics for your model
metrics = target_model.val()

## Test the Model

With a trained model ready, we can run inference on a few example images of containers to see how our model performs. Let's do it!

In [ ]:
example_image = "./containers_labeled/train/images/0c59799a5b2b95c8b53df911264676f71b0e59c8_jpg.rf.c7af7b3a1332460b5495be354468fd65.jpg"

pred = target_model.predict(example_image, conf=0.01)

res_plotted = pred[0].plot()
image_rgb = cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB)

sv.plot_image(image_rgb)

## (Optional) Run Inference on a Video

Using `supervision`, we can run inference on a video to detect containers throughout the video. In the code below, we'll use a video of shipping containers and annotate all containers. We'll also show a count of the number of containers present in each frame.

In [ ]:
import supervision as sv
import numpy as np

box_annotator = sv.BoxAnnotator(thickness=2, text_thickness=4, text_scale=2)

# classes are taken from the class name values in the ontology we specified earlier
classes = "container"

VIDEO = f"{HOME}/video.mp4"
OUTPUT = f"{HOME}/output.mp4"

def process_frame(frame: np.ndarray, i: int) -> np.ndarray:
    print(f"Processing frame {i}")
    predictions = target_model(frame).json()
    detections = sv.Detections.from_yolov8(roboflow_result = predictions, class_list = classes)
    detections = detections[(detections.confidence > 0.3)]

    labels = [
        f"{classes[class_id]} {confidence:0.2f}"
        for _, _, confidence, class_id, _
        in detections
    ]

    frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)

    return frame

In [ ]:
generator = sv.get_video_frames_generator(VIDEO)

sv.process_video(
    source_path=VIDEO,
    target_path=OUTPUT,
    callback=lambda frame, i: process_frame(frame, i)
)

print(f"Output saved to {OUTPUT}.")

Open the video you saved to see the result of the model on the video.

## 🏆 Congratulations

### Learning Resources

Roboflow has produced many resources that you may find interesting as you advance your knowledge of computer vision:

- [Roboflow Notebooks](https://github.com/roboflow/notebooks): A repository of over 20 notebooks that walk through how to train custom models with a range of model types, from YOLOv7 to SegFormer.
- [Roboflow YouTube](https://www.youtube.com/c/Roboflow): Our library of videos featuring deep dives into the latest in computer vision, detailed tutorials that accompany our notebooks, and more.
- [Roboflow Discuss](https://discuss.roboflow.com/): Have a question about how to do something on Roboflow? Ask your question on our discussion forum.
- [Roboflow Models](https://roboflow.com/models): Learn about state-of-the-art models and their performance. Find links and tutorials to guide your learning.

### Convert data formats

Roboflow provides free utilities to convert data between dozens of popular computer vision formats. Check out [Roboflow Formats](https://roboflow.com/formats) to find tutorials on how to convert data between formats in a few clicks.

### Connect computer vision to your project logic

[Roboflow Templates](https://roboflow.com/templates) is a public gallery of code snippets that you can use to connect computer vision to your project logic. Code snippets range from sending emails after inference to measuring object distance between detections.